## 1. API request

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
response = requests.get('https://www.ditp.go.th/wp-json/ditp/v1/posts?offset=0&limit=20')
j=response.json()

In [12]:
data=[]
for i in j['data']:
    d={
        'title':i['Title'],
        'publishdate':i['PublishDate'],
        'url':i['URL'],
        'pdf':i['PDF'],
    }
    data.append(d)
df=pd.DataFrame.from_dict(data)
df2=df[df['pdf']!=df['pdf']]
df2

,title,publishdate,url,pdf
15,สถานการณ์เศรษฐกิจการค้าการลงทุน: บริษัท และอง...,2566-11-01 21:51:16,https://www.ditp.go.th/post/151577,None
16,รายงานสถานการณ์เศรษฐกิจการค้าระหว่างประเทศของส...,2566-11-01 14:27:03,https://www.ditp.go.th/post/151531,None
17,อินเดียกำลังจะเป็นตลาดที่ใหญ่ที่สุดสำหรับเพชรแ...,2566-11-01 14:22:17,https://www.ditp.go.th/post/151538,None


## 2.Scrap content from a website

In [15]:
url='https://www.ditp.go.th/post/151577'
html=requests.get(url).text
soup = BeautifulSoup(html, "html.parser")
texts = soup.find_all('p')
l=[text.get_text() for text in texts]
content="\n".join(l)
content

'บริษัทจำหน่ายของขวัญวันคริสต์มาสสำหรับองค์กรจากเดนมาร์ก บริษัท Gavefabrikken ให้ข้อมูลว่า บริษัทฯ สามารถทำยอดขายเติบโตสูงสุดเป็นประวัติการณ์ตลอดช่วง 14 ปีติดต่อกัน\nบริษัท Gavefabrikken คือ บริษัทซัพพลายเออร์จำหน่ายของขวัญคริสต์มาสสำหรับองค์กรรายใหญ่ที่สุดในกลุ่มประเทศนอร์ดิก ประธานบริหารตำแหน่ง CEO ให้ข้อมูลว่า ถึงแม้ว่าบริษัทจะเผชิญกับความท้าทายในตลาดสวีเดน แต่ผลประกอบการโดยรวมเป็นที่น่าพอใจ โดยปี 2565 สามารถทำยอดขายได้ 356 ล้านโครนเดนมาร์ก (ประมาณ 50.51 ล้านเหรียญสหรัฐ) ขยายตัวร้อยละ 9 เมื่อเทียบกับปีที่ผ่านมา\nบริษัท Gavefabrikken ก่อตั้งในปี 2551 โดยผู้บริหารคนปัจจุบัน จำหน่ายของขวัญให้กับบริษัท และองค์กรต่างๆ ในเดนมาร์กมากกว่า 1 ล้านชิ้น นอกจากตลาดเดนมาร์กแล้ว บริษัทฯ ยังขยายตลาดไปยังนอร์เวย์ และสวีเดน ปัจจุบันมีพนักงาน 80 ราย\nจากข้อมูลของบริษัทฯ การเติบโตของยอดขายนี้มาจาก 2 ปัจจัยหลัก คือ งบประมาณของขวัญของบริษัทต่างๆ ที่เพิ่มขึ้น และการประสบความสำเร็จในการขยายส่วนแบ่งตลาดในเดนมาร์ก โดยในปี 2566 นี้ ลูกค้าให้ความสำคัญกับคุณภาพของสินค้ามากขึ้น สินค้าที่โดดเด่นด้านการออกแบบ เช่น

## 3.Using Monad

In [16]:
import requests

class Result:
    def __init__(self, value=None, error=None):
        self.value = value
        self.error = error

    @classmethod
    def of(cls, func, *args, **kwargs):
        try:
            return cls(value=func(*args, **kwargs))
        except Exception as e:
            return cls(error=str(e))

    @staticmethod
    def success(x):
        return Result(value=x)

    @staticmethod
    def failure(error):
        return Result(error=error)

    def is_error(self):
        return self.error is not None
    
    def bind(self, func):
        if self.is_error():
            return self
        return Result.of(func, self.value)

    def __repr__(self):
        if self.is_error():
            return f"Error({self.error})"
        return f"Success({self.value})"
    
    
def resultify(func):
    def wrapper(*args, **kwargs):
        return Result.of(func, *args, **kwargs)
    return wrapper

In [17]:
@resultify
def get_content(url):
    html=requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    texts = soup.find_all('p')
    l=[text.get_text() for text in texts]
    content="\n".join(l)
    return content

In [20]:
df['url'].apply(get_content)

0     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
1     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
2     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
3     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
4     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
5     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
6     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
7     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
8     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
9     Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
10    Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
11    Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
12    Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
13    Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
14    Success(สมัครรับข้อมูลข่าวสาร\nกรมส่งเสริมการค...
15    Success(บริษัทจำหน่ายของขวัญวันคริสต์มาสสำหรับ...
16    Success(รัฐบาลเมียนมาออกประกาศเมื่อวันที่ 25 ต...
17    Success( อินเดียกำลังจะเป็นตลาดที่ใหญ่ที่ส

In [25]:
df3=df2
df3.loc[:,'content']=df2['url'].apply(get_content)

In [26]:
df3

,title,publishdate,url,pdf,content
15,สถานการณ์เศรษฐกิจการค้าการลงทุน: บริษัท และอง...,2566-11-01 21:51:16,https://www.ditp.go.th/post/151577,None,Success(บริษัทจำหน่ายของขวัญวันคริสต์มาสสำหรับ...
16,รายงานสถานการณ์เศรษฐกิจการค้าระหว่างประเทศของส...,2566-11-01 14:27:03,https://www.ditp.go.th/post/151531,None,Success(รัฐบาลเมียนมาออกประกาศเมื่อวันที่ 25 ต...
17,อินเดียกำลังจะเป็นตลาดที่ใหญ่ที่สุดสำหรับเพชรแ...,2566-11-01 14:22:17,https://www.ditp.go.th/post/151538,None,Success( อินเดียกำลังจะเป็นตลาดที่ใหญ่ที่สุดสำ...
